In [1]:
import os
import cv2
import pickle
import numpy as np
import pdb
import requests
from collections import defaultdict
import random 
import time
import warnings
import matplotlib.pyplot as plt
import math


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from tqdm import *

from functools import wraps
from time import time as _timenow 
from sys import stderr
print('its done') 


its done


## Load CIFAR-10 Data

In [2]:
def load_cifar():
    
    trn_data, trn_labels, tst_data, tst_labels = [], [], [], []
    def unpickle(file):
        with open(file, 'rb') as fo:
            data = pickle.load(fo, encoding='latin1')
        return data
    
    for i in trange(1):
        batchName = './data/data_batch_{0}'.format(i + 1)
        unpickled = unpickle(batchName)
        trn_data.extend(unpickled['data'])
        trn_labels.extend(unpickled['labels'])
    unpickled = unpickle('./data/test_batch')
    tst_data.extend(unpickled['data'])
    tst_labels.extend(unpickled['labels'])
    return trn_data, trn_labels, tst_data, tst_labels
trn_data, trn_labels, tst_data, tst_labels = load_cifar()
print('itsdone')


  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './data/data_batch_1'

## Image preprocessing

In [ ]:
def preprocess(train,test):
    scaler = preprocessing.StandardScaler().fit(train)
    train_data = scaler.transform(train)
    test_data = scaler.transform(test)
    ''' pre-processes the given image
        performs mean normalization and other such operations'''
    return train_data,test_data

## Dimensionality reduction using PCA

In [ ]:
def reduce_dim(**kwargs):
    ''' performs dimensionality reduction'''
    if kwargs['method'] == 'lda':
        lda=LDA(n_components=kwargs['no_of_components'])
        lda = lda.fit(kwargs['train'],kwargs['y'])
        train_new = lda.transform(kwargs['train'])
        test_new = lda.transform(kwargs['test'])
        return train_new, test_new
    if kwargs['method'] == 'pca':
        pca = PCA(n_components=kwargs['no_of_components'])
        pca = pca.fit(kwargs['train'])
        train_new = pca.transform(kwargs['train'])
        test_new = pca.transform(kwargs['test'])
        return train_new, test_new

## Classification using kernel SVM

In [ ]:
def classify(X, y, **kwargs):
    print('it came')
    ''' trains a classifier by taking input features
        and their respective targets and returns the trained model'''
    if kwargs['method'] == 'LR':
        lr = LogisticRegression(C=math.exp(kwargs['c']),solver='lbfgs',multi_class='multinomial',class_weight ='balanced',max_iter=kwargs['max_iterations'])
        lr.fit(X, y)
        print('it is classified')
        return lr

## Evaluation 

In [ ]:
def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='micro')
    acc = accuracy_score(target, predicted)
    return f1, acc

In [ ]:
def test(**kwargs):
    '''takes test data and trained classifier model,
    performs classification and prints accuracy and f1-score'''
    if kwargs['method'] == 'LR':
        lr = kwargs['model']
        testdata = kwargs['testdata']
        out = lr.predict(testdata)
        return out

In [ ]:
def main():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_test, y_train, y_test = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    #print(trn_labels)
    #pre_test,pre_train=preprocess(X_train,X_test)
    pre_test,pre_train=X_train,X_test
    i=25;
    f1_array=[]
    acc_array=[]
    i_array=[]
    while(i<500):
    #train_new,test_new=reduce_dim(method='lda',train=pre_test,test=pre_train,y=y_train)
        train_new,test_new=reduce_dim(method='pca',train=pre_test,test=pre_train,no_of_components=i)
        #lr=classify(pre_test,trn_labels,method='LR')
        lr=classify(train_new,y_train,method='LR',max_iterations=2000,c=5)
        out= test(method='LR',model=lr,testdata=test_new)
        f1,acc=evaluate(y_test,out)
        f1_array.append(f1)
        acc_array.append(acc)
        i_array.append(i)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f1, acc))
        print(i)
        i=i+25
    
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
        '''
    plt.plot(i_array,acc_array)
    plt.xlabel('No. of pca components') 
    plt.ylabel('Accuracy') 
    plt.show()
    

In [ ]:
def main2():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_test, y_train, y_test = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    #print(trn_labels)
    #pre_test,pre_train=preprocess(X_train,X_test)
    pre_test,pre_train=X_train,X_test
    
    i=1000;
    f1_array=[]
    acc_array=[]
    i_array=[]
    while(i<3000):
    #train_new,test_new=reduce_dim(method='lda',train=pre_test,test=pre_train,y=y_train)
        train_new,test_new=reduce_dim(method='pca',train=pre_test,test=pre_train,no_of_components=150)
        #lr=classify(pre_test,trn_labels,method='LR')
        lr=classify(train_new,y_train,method='LR',max_iterations=i,c=5)
        out= test(method='LR',model=lr,testdata=test_new)
        f1,acc=evaluate(y_test,out)
        f1_array.append(f1)
        acc_array.append(acc)
        i_array.append(i)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f1, acc))
        print(i)
        i=i+200
    plt.plot(i_array,acc_array)
    plt.xlabel('Max number of iterations') 
    plt.ylabel('Accuracy') 
    plt.show()
    i=1000;

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
main2()

In [ ]:
def main3():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_test, y_train, y_test = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    #print(trn_labels)
    #pre_test,pre_train=preprocess(X_train,X_test)
    pre_test,pre_train=X_train,X_test
    
    i=25;
    f1_array=[]
    acc_array=[]
    i_array=[]
    while(i<500):
        train_new,test_new=reduce_dim(method='lda',train=pre_test,test=pre_train,y=y_train,no_of_components=i)
        #train_new,test_new=reduce_dim(method='pca',train=pre_test,test=pre_train,no_of_components=150)
        #train_new,test_new=pre_train,pre_test
        #lr=classify(pre_test,trn_labels,method='LR')
        lr=classify(train_new,y_train,method='LR',max_iterations=1000,c=5)
        out= test(method='LR',model=lr,testdata=test_new)
        f1,acc=evaluate(y_test,out)
        f1_array.append(f1)
        acc_array.append(acc)
        i_array.append(i)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f1, acc))
        print(i)
        i=i+50
    plt.plot(i_array,acc_array)
    plt.xlabel('NO of lda components') 
    plt.ylabel('Accuracy') 
    plt.show()
    i=1000;

In [ ]:
main3()

In [ ]:
def main4():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_test, y_train, y_test = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    #print(trn_labels)
    #pre_test,pre_train=preprocess(X_train,X_test)
    pre_test,pre_train=X_train,X_test
    
    i=1;
    f1_array=[]
    acc_array=[]
    i_array=[]
    while(i<7):
    #train_new,test_new=reduce_dim(method='lda',train=pre_test,test=pre_train,y=y_train)
        train_new,test_new=reduce_dim(method='pca',train=pre_test,test=pre_train,no_of_components=150)
        #lr=classify(pre_test,trn_labels,method='LR')
        lr=classify(train_new,y_train,method='LR',max_iterations=2400,c=i)
        out= test(method='LR',model=lr,testdata=test_new)
        f1,acc=evaluate(y_test,out)
        f1_array.append(f1)
        acc_array.append(acc)
        i_array.append(i)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f1, acc))
        print(i)
        i=i+1
    plt.plot(i_array,acc_array)
    plt.xlabel('regulirisation strength') 
    plt.ylabel('Accuracy') 
    plt.show()